### Reference

[Mastering Oracle+Python by Przemyslaw Piotrowski](https://www.oracle.com/technetwork/articles/dsl/mastering-oracle-python-1391323.html)

https://www.oracle.com/technetwork/articles/dsl/prez-python-queries-101587.html

### Setup

Install cx_Oracle on Windows

https://www.cs.utexas.edu/~scohen/cs327e_spr15/cx_Oracle/windows.html

use Anaconda python 3.6


download cx_Oracle from https://pypi.org/project/cx_Oracle/#files

[cx_Oracle-6.4.1-cp36-cp36m-win_amd64.whl](https://files.pythonhosted.org/packages/ae/ee/5e9704b60f99540a6b6f527f355b5e7025ba3f637f31566c5021e6e337c5/cx_Oracle-6.4.1-cp36-cp36m-win_amd64.whl) (164.8 kB)  Copy SHA256 hash SHA256	Wheel	Jul 9, 2018

install wheel

python -m pip install --upgrade pip setuptools wheel

pip install cx_Oracle-6.4.1-cp36-cp36m-win_amd64.whl

Install OracleXE

unlock account=hr/hr

other user passwords

system/oracle101
sys/oracle101
hr/hr

In [1]:
import cx_Oracle

In [2]:
import pandas as pd

In [3]:
con = cx_Oracle.connect('hr/hr@127.0.0.1/xe')

In [4]:
print(con.version)

# 11.2.0.2.0

11.2.0.2.0


### Basic 


https://www.oracle.com/technetwork/articles/dsl/prez-python-queries-101587.html

#### Connection

In [8]:
user_name = "hr"
pwd = "hr"
hostname = "localhost"
port = "1521"
service_name = "XE"
conn_str = "{0}/{1}@{2}:{3}/{4}".format(user_name, pwd, hostname, port, service_name)


In [9]:
conn_str

'hr/hr@localhost:1521/XE'

In [10]:
# 2 ways to create connection
conn1 = cx_Oracle.connect(conn_str)

In [11]:
dsn_tns = cx_Oracle.makedsn(hostname, port, service_name)
dsn_tns

'(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=XE)))'

In [12]:
conn2 = cx_Oracle.connect(user_name, pwd, dsn_tns)

In [13]:
# get database version
print(conn2.version)

11.2.0.2.0


In [15]:
conn1.dsn

'localhost:1521/XE'

In [16]:
conn2.dsn

'(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=XE)))'

#### Cursor

Application logic often requires clearly distinguishing the stages of processing a statement issued against the database. This will help understand performance bottlenecks better and allow writing faster, optimized code. The three stages of processing a statement are: 

* Parse (optional)
    - cx_Oracle.Cursor.parse([statement]) 
Not really required to be called because SQL statements are automatically parsed at the Execute stage. It can be used to validate statements before executing them. When an error is detected in such a statement, a DatabaseError exception is raised with a corresponding error message, most likely "ORA-00900: invalid SQL statement, ORA-01031: insufficient privileges or ORA-00921: unexpected end of SQL command."

* Execute
    - cx_Oracle.Cursor.execute(statement, [parameters], **keyword_parameters)
This method can accept a single argument - a SQL statement - to be run directly against the database. Bind variables assigned through the parameters or keyword_parameters arguments can be specified as a dictionary, sequence, or a set of keyword arguments. If dictionary or keyword arguments are supplied then the values will be name-bound. If a sequence is given, the values will be resolved by their position. This method returns a list of variable objects if it is a query, and None when it's not.

    - cx_Oracle.Cursor.executemany(statement, parameters)
Especially useful for bulk inserts because it can limit the number of required Oracle execute operations to just a single one. For more information about how to use it please see the "Many at once" section below.
 
* Fetch (optional)—Only used for queries (because DDL and DCL statements don't return results). On a cursor that didn't execute a query, these methods will raise an InterfaceError exception.

    - cx_Oracle.Cursor.fetchall() 
Fetches all remaining rows of the result set as a list of tuples. If no more rows are available, it returns an empty list. Fetch actions can be fine-tuned by setting the arraysize attribute of the cursor which sets the number of rows to return from the database in each underlying request. The higher setting of arraysize, the fewer number of network round trips required. The default value for arraysize is 1.

    - cx_Oracle.Cursor.fetchmany([rows_no]) 
Fetches the next rows_no rows from the database. If the parameter isn't specified it fetches the arraysize number of rows. In situations where rows_no is greater than the number of fetched rows, it simply gets the remaining number of rows.

    - cx_Oracle.Cursor.fetchone() 
Fetches a single tuple from the database or none if no more rows are available.

In [17]:
cur = conn1.cursor()

In [19]:
cur.parse("select * from emp")

DatabaseError: ORA-00942: table or view does not exist

In [20]:
cur.parse("select * from employees")

In [33]:
column_data_types = cur.execute("select * from employees")

In [34]:
column_data_types

<cx_Oracle.Cursor on <cx_Oracle.Connection to hr@localhost:1521/XE>>

In [37]:
# cx_Oracle.Cursor.execute* family of methods returns column data types for queries. 
# These are lists of Variable objects
cur.description

[('EMPLOYEE_ID', cx_Oracle.NUMBER, 7, None, 6, 0, 0),
 ('FIRST_NAME', cx_Oracle.STRING, 20, 20, None, None, 1),
 ('LAST_NAME', cx_Oracle.STRING, 25, 25, None, None, 0),
 ('EMAIL', cx_Oracle.STRING, 25, 25, None, None, 0),
 ('PHONE_NUMBER', cx_Oracle.STRING, 20, 20, None, None, 1),
 ('HIRE_DATE', cx_Oracle.DATETIME, 23, None, None, None, 0),
 ('JOB_ID', cx_Oracle.STRING, 10, 10, None, None, 0),
 ('SALARY', cx_Oracle.NUMBER, 12, None, 8, 2, 1),
 ('COMMISSION_PCT', cx_Oracle.NUMBER, 6, None, 2, 2, 1),
 ('MANAGER_ID', cx_Oracle.NUMBER, 7, None, 6, 0, 1),
 ('DEPARTMENT_ID', cx_Oracle.NUMBER, 5, None, 4, 0, 1)]

In [22]:
cur.fetchone()

(100,
 'Steven',
 'King',
 'SKING',
 '515.123.4567',
 datetime.datetime(2003, 6, 17, 0, 0),
 'AD_PRES',
 24000.0,
 None,
 None,
 90)

In [27]:
rows_3 = cur.fetchmany(3)

In [28]:
type(rows_3)

list

In [29]:
rows_3

[(104,
  'Bruce',
  'Ernst',
  'BERNST',
  '590.423.4568',
  datetime.datetime(2007, 5, 21, 0, 0),
  'IT_PROG',
  6000.0,
  None,
  103,
  60),
 (105,
  'David',
  'Austin',
  'DAUSTIN',
  '590.423.4569',
  datetime.datetime(2005, 6, 25, 0, 0),
  'IT_PROG',
  4800.0,
  None,
  103,
  60),
 (106,
  'Valli',
  'Pataballa',
  'VPATABAL',
  '590.423.4560',
  datetime.datetime(2006, 2, 5, 0, 0),
  'IT_PROG',
  4800.0,
  None,
  103,
  60)]

In [30]:
rows_all = cur.fetchall()

In [31]:
rows_all[:1]

[(107,
  'Diana',
  'Lorentz',
  'DLORENTZ',
  '590.423.5567',
  datetime.datetime(2007, 2, 7, 0, 0),
  'IT_PROG',
  4200.0,
  None,
  103,
  60)]

In [32]:
len(rows_all)

100

#### Bind Variable 

In [39]:
select_stmt = """
    SELECT * FROM employees 
    WHERE department_id= :dept_id 
    AND salary> :sal
"""

In [40]:
cur.parse(select_stmt)

In [41]:
named_params = {'dept_id':50, 'sal':1000}

In [43]:
query1 = cur.execute(select_stmt, named_params)

In [56]:
query1.description

[('EMPLOYEE_ID', cx_Oracle.NUMBER, 7, None, 6, 0, 0),
 ('FIRST_NAME', cx_Oracle.STRING, 20, 20, None, None, 1),
 ('LAST_NAME', cx_Oracle.STRING, 25, 25, None, None, 0),
 ('EMAIL', cx_Oracle.STRING, 25, 25, None, None, 0),
 ('PHONE_NUMBER', cx_Oracle.STRING, 20, 20, None, None, 1),
 ('HIRE_DATE', cx_Oracle.DATETIME, 23, None, None, None, 0),
 ('JOB_ID', cx_Oracle.STRING, 10, 10, None, None, 0),
 ('SALARY', cx_Oracle.NUMBER, 12, None, 8, 2, 1),
 ('COMMISSION_PCT', cx_Oracle.NUMBER, 6, None, 2, 2, 1),
 ('MANAGER_ID', cx_Oracle.NUMBER, 7, None, 6, 0, 1),
 ('DEPARTMENT_ID', cx_Oracle.NUMBER, 5, None, 4, 0, 1)]

In [57]:
col_names = [c[0] for c in query1.description]

In [58]:
col_names

['EMPLOYEE_ID',
 'FIRST_NAME',
 'LAST_NAME',
 'EMAIL',
 'PHONE_NUMBER',
 'HIRE_DATE',
 'JOB_ID',
 'SALARY',
 'COMMISSION_PCT',
 'MANAGER_ID',
 'DEPARTMENT_ID']

In [48]:
rows = cur.fetchall()

In [50]:
len(rows)

45

In [59]:
df = pd.DataFrame(columns=col_names, data=rows)

In [60]:
df.head()

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
0,120,Matthew,Weiss,MWEISS,650.123.1234,2004-07-18,ST_MAN,8000.0,None,100,50
1,121,Adam,Fripp,AFRIPP,650.123.2234,2005-04-10,ST_MAN,8200.0,None,100,50
2,122,Payam,Kaufling,PKAUFLIN,650.123.3234,2003-05-01,ST_MAN,7900.0,None,100,50
3,123,Shanta,Vollman,SVOLLMAN,650.123.4234,2005-10-10,ST_MAN,6500.0,None,100,50
4,124,Kevin,Mourgos,KMOURGOS,650.123.5234,2007-11-16,ST_MAN,5800.0,None,100,50


In [64]:
cur.execute('SELECT * FROM locations WHERE country_id=:1 AND city=:2', ('US', 'Seattle'))

<cx_Oracle.Cursor on <cx_Oracle.Connection to hr@localhost:1521/XE>>

In [65]:
cur.fetchmany(6)

[(1700, '2004 Charade Rd', '98199', 'Seattle', 'Washington', 'US')]

#### Many at Once

In [66]:
create_table = """
    CREATE TABLE python_modules (
    module_name VARCHAR2(50) NOT NULL,
    file_path VARCHAR2(300) NOT NULL
)
"""
cur.execute(create_table)

In [67]:
cur.execute("select * from python_modules")
rows = cur.fetchall()
rows

[]

In [69]:
from sys import modules
M = []
for m_name, m_info in modules.items():
    try:
        M.append((m_name, m_info.__file__))
    except AttributeError:
        pass

len(M)

1100

In [70]:
M[:3]

[('_frozen_importlib', 'C:\\Anaconda3\\lib\\importlib\\_bootstrap.py'),
 ('_frozen_importlib_external',
  'C:\\Anaconda3\\lib\\importlib\\_bootstrap_external.py'),
 ('encodings', 'C:\\Anaconda3\\lib\\encodings\\__init__.py')]

In [71]:
cur.prepare("INSERT INTO python_modules(module_name, file_path) VALUES (:1, :2)")
cur.executemany(None, M)
conn1.commit()

In [72]:
cur.execute("SELECT COUNT(*) FROM python_modules")
cur.fetchone()

(1100,)

In [73]:
# drop table

#cur.execute("DROP TABLE python_modules PURGE")

#### Pandas

In [74]:
df = pd.read_sql("""select * from python_modules""", con=conn1)

In [75]:
df.head()

,MODULE_NAME,FILE_PATH
0,pygments,C:\Anaconda3\lib\site-packages\pygments\__init...
1,pygments.util,C:\Anaconda3\lib\site-packages\pygments\util.py
2,IPython.utils.py3compat,C:\Anaconda3\lib\site-packages\IPython\utils\p...
3,IPython.utils.encoding,C:\Anaconda3\lib\site-packages\IPython\utils\e...
4,IPython.core.excolors,C:\Anaconda3\lib\site-packages\IPython\core\ex...


In [76]:
# select columns
df.head(10)[['MODULE_NAME','FILE_PATH']]

,MODULE_NAME,FILE_PATH
0,pygments,C:\Anaconda3\lib\site-packages\pygments\__init...
1,pygments.util,C:\Anaconda3\lib\site-packages\pygments\util.py
2,IPython.utils.py3compat,C:\Anaconda3\lib\site-packages\IPython\utils\p...
3,IPython.utils.encoding,C:\Anaconda3\lib\site-packages\IPython\utils\e...
4,IPython.core.excolors,C:\Anaconda3\lib\site-packages\IPython\core\ex...
5,IPython.testing,C:\Anaconda3\lib\site-packages\IPython\testing...
6,IPython.testing.skipdoctest,C:\Anaconda3\lib\site-packages\IPython\testing...
7,pdb,C:\Anaconda3\lib\pdb.py
8,cmd,C:\Anaconda3\lib\cmd.py
9,code,C:\Anaconda3\lib\code.py


In [79]:
for ix in range(5):
    print(df.iloc[ix]['FILE_PATH'])

C:\Anaconda3\lib\site-packages\pygments\__init__.py
C:\Anaconda3\lib\site-packages\pygments\util.py
C:\Anaconda3\lib\site-packages\IPython\utils\py3compat.py
C:\Anaconda3\lib\site-packages\IPython\utils\encoding.py
C:\Anaconda3\lib\site-packages\IPython\core\excolors.py


#### connect to CRMAT

In [11]:
user_name = "intfc_gcm_dqms"
pwd = "Xuvnkdyx0"
hostname = "ubadx4013-vip.us.oracle.com"
port = "1553"
service_name = "crmat.us.oracle.com"
conn_str = "{0}/{1}@{2}:{3}/{4}".format(user_name, pwd, hostname, port, service_name)

In [12]:
conn_str

'intfc_gcm_dqms/Xuvnkdyx0@ubadx4013-vip.us.oracle.com:1553/crmat.us.oracle.com'

In [13]:
crmat = cx_Oracle.connect(conn_str)

In [14]:
sql_str = """
    select p.party_id,p.party_name,p.status 
    from fusion.hz_parties p where rownum < 15
"""

df = pd.read_sql(sql_str, con=crmat)

In [15]:
df

,PARTY_ID,PARTY_NAME,STATUS
0,300000000463755,None,A
1,300000000463763,Other-Local,A
2,300000000463771,IBM,A
3,300000000463779,HP,A
4,300000000463787,SalesForce.com,A
5,300000000463795,EMC CORPORATION,A
6,300000000463803,TERADATA,A
7,300000000463811,NetApp,A
8,300000000463823,SAP Sybase,A
9,300000000463835,SAP Business Objects,A


### Managing database transactions

https://www.oracle.com/technetwork/articles/dsl/prez-transactions-lobs-089563.html
    
It's All About ACID
A database transaction is a logical group of statements characterized by four essential properties:

- Atomicity: all of the operations succeed or fail as a whole
- Consistency: committing transaction cannot result in data corruption
- Isolation: other transactions remain unaware of transaction's execution
- Durability: operations committed within transaction will prevail database crash.

In [82]:
import cx_Oracle
 
class HR:
  def __enter__(self):
    self.__db = cx_Oracle.Connection("hr/hr@localhost:1521/XE")
    self.__cursor = self.__db.cursor()
    return self 
  
  def __exit__(self, type, value, traceback): 
    self.__db.close()
  
  def swapDepartments(self, employee_id1, employee_id2):
    assert employee_id1!=employee_id2
    
    select_sql = """select employee_id, department_id from employees
      where employee_id in (:1, :2)"""
    update_sql = "update employees set department_id=:1 where employee_id=:2"

    self.__db.begin()
    self.__cursor.execute(select_sql, (employee_id1, employee_id2))
    D = dict(self.__cursor.fetchall())
    self.__cursor.execute(update_sql, (D[employee_id2], employee_id1))
    self.__cursor.execute(update_sql, (D[employee_id1], employee_id2))
    self.__db.commit()
 
  def raiseSalary(self, employee_ids, raise_pct):
    update_sql = "update employees set salary=salary*:1 where employee_id=:2"
    
    self.__db.begin()

    for employee_id in employee_ids:
      try:
        self.__cursor.execute(update_sql, [1+raise_pct/100, employee_id])
        assert self.__cursor.rowcount==1
      except AssertionError:
        self.__db.rollback()
        raise (Warning, "invalid employee_id (%s)" % employee_id)

    self.__db.commit()
 
if __name__ == "__main__":
  with HR() as hr:
    hr.swapDepartments(106, 116)
    hr.raiseSalary([102, 106, 116], 20)

In [86]:
cur.execute("select employee_id, department_id, salary from employees where employee_id in (102, 106,116)")

cur.fetchall()

<cx_Oracle.Cursor on <cx_Oracle.Connection to hr@localhost:1521/XE>>

In [88]:
with HR() as hr:
    hr.swapDepartments(106, 116)
    hr.raiseSalary([102, 106, 116], 20)

In [ ]:
# verify after changes

In [89]:
cur.execute("select employee_id, department_id, salary from employees where employee_id in (102, 106,116)")

cur.fetchall()

[(102, 90, 24480.0), (106, 60, 6912.0), (116, 30, 4176.0)]

In [90]:
with HR() as hr:
    hr.raiseSalary([102, 106, 116], -20)

In [91]:
cur.execute("select employee_id, department_id, salary from employees where employee_id in (102, 106,116)")

cur.fetchall()

[(102, 90, 19584.0), (106, 60, 5529.6), (116, 30, 3340.8)]

### Handling Large Objects

https://www.oracle.com/technetwork/articles/dsl/prez-transactions-lobs-089563.html
    
    
When it comes to data types available for table columns in the Oracle Database, VARCHAR2 can only store values up to 4000 bytes. This is where Large Objects chime in with its ability to store large data such as text, images, videos and other multimedia formats. And with what you can hardly call a limit—a LOB's maximum size is as much as 128 terabytes (as of 11g Release 2).


There are several types of LOBs available in Oracle Database: 
- Binary Large Object (BLOB), 
- Character Large Object (CLOB), 
- National Character Set Large Object (NCLOB) and 
- External Binary File (BFILE). 

The latter one is used for accessing external operating system files in read-only mode while all the other ones enable storage of large data inside the database, in either persistent or temporary mode. Each LOB consists of an actual value and a small locator that points to the value. Passing a LOB around often simply means passing a LOB locator.


At any given time, a LOB can be in one of the three defined states: NULL, empty or populated. This resembles the behavior of regular VARCHAR columns in other RDBMS engines (not treating empty string as NULL). And lastly there are several restrictions with the major ones being that LOBs:

- Cannot be a primary key
- Cannot be part of a cluster
- Cannot be used in conjunction with DISTINCT, ORDER BY and GROUP BY clauses

In [92]:
[i for i in dir(cx_Oracle) if i.endswith('LOB') or i=='BFILE'] 

['BFILE', 'BLOB', 'CLOB', 'LOB', 'NCLOB']

In [93]:
clob = cur.var(cx_Oracle.CLOB)
clob.setvalue(0, '#'*2**10) 

In [95]:
print(clob.getvalue(0))

########################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################

In [96]:
# see cx_oracle_lob.py and c:/tmp/example.txt
cur.execute("SELECT bf FROM v_lobs WHERE bf IS NOT NULL") 

<cx_Oracle.Cursor on <cx_Oracle.Connection to hr@localhost:1521/XE>>

In [97]:
bf, = cur.fetchone()

In [98]:
type(bf)

cx_Oracle.LOB

In [107]:
bfile_data = bf.read()

In [108]:
type(bfile_data)

bytes

In [110]:
# decode the bytes object to produce a string:
print(bfile_data.decode("utf-8"))

LOB as in Large Objects
When it comes to data types available for table columns in the Oracle Database, VARCHAR2 can only store values up to 4000 bytes. This is where Large Objects chime in with its ability to store large data such as text, images, videos and other multimedia formats. And with what you can hardly call a limit—a LOB's maximum size is as much as 128 terabytes (as of 11g Release 2).

There are several types of LOBs available in Oracle Database: Binary Large Object (BLOB), Character Large Object (CLOB), National Character Set Large Object (NCLOB) and External Binary File (BFILE). The latter one is used for accessing external operating system files in read-only mode while all the other ones enable storage of large data inside the database, in either persistent or temporary mode. Each LOB consists of an actual value and a small locator that points to the value. Passing a LOB around often simply means passing a LOB locator.

At any given time, a LOB can be in one of the three

### Stored Procedure

[Mastering Oracle+Python, Part 5: Stored Procedures, Programming Python
by Przemyslaw Piotrowski](https://www.oracle.com/technetwork/articles/prez-stored-proc-084100.html)

In [18]:
xe_conn_str = "hr/hr@//localhost:1521/XE"

In [ ]:
db = cx_Oracle.connect(xe_conn_str)
cursor = db.cursor()

In [19]:
lst = cursor.arrayvar(cx_Oracle.NUMBER, [1, 2, 3])

sum_result = cursor.callfunc("pkg_arrayvar.sum", cx_Oracle.NUMBER, [lst])
assert sum_result==6

In [20]:
sum_result

6.0

In [24]:
lst2 = list(range(10))

In [25]:
sum_result = cursor.callfunc("pkg_arrayvar.sum", cx_Oracle.NUMBER, [lst2])
sum_result

45.0

In [17]:
class HR:
  def __enter__(self):
    self.__db = cx_Oracle.Connection("hr/hr@//localhost:1521/XE")
    self.__cursor = self.__db.cursor()
    return self 
  
  def __exit__(self):
    self.__cursor.close()
    self.__db.close()
 
  def add_department(self, p_department_name, p_manager_id, p_location_id):
    l_department_id = self.__cursor.var(cx_Oracle.NUMBER)
    self.__cursor.callproc("PKG_HR.ADD_DEPARTMENT",
      [l_department_id, p_department_name, p_manager_id, p_location_id]) 
  
    # there are no OUT parameters in Python, regular return here
    return l_department_id    
  
  def get_employee_count(self, p_department_id):
    l_count = self.__cursor.callfunc("PKG_HR.GET_EMPLOYEE_COUNT",
      cx_Oracle.NUMBER, [p_department_id]) 
    return l_count 
  
  def find_employees(self, p_query):
    # as it comes to all complex types we need to tell Oracle Client
    # what type to expect from an OUT parameter
    l_cur = self.__cursor.var(cx_Oracle.CURSOR)
    l_query, l_emp = self.__cursor.callproc("PKG_HR.FIND_EMPLOYEES", [p_query, l_cur]) 
    return list(l_emp)

  def find_employee_names(self, p_query):
    # as it comes to all complex types we need to tell Oracle Client
    # what type to expect from an OUT parameter
    l_cur = self.__cursor.var(cx_Oracle.CURSOR)
    l_query, l_emp = self.__cursor.callproc("PKG_HR.FIND_EMPLOYEE_NAMES", [p_query, l_cur]) 
    return list(l_emp)

In [18]:
hr1 = HR()
hr1=hr1.__enter__()

In [19]:
hr1.get_employee_count(100)

6.0

In [20]:
df = hr1.find_employees("K")

In [21]:
len(df)

23

In [14]:
df[0]

(100,
 'Steven',
 'King',
 'SKING',
 '515.123.4567',
 datetime.datetime(2003, 6, 17, 0, 0),
 'AD_PRES',
 24000.0,
 None,
 None,
 90)

In [15]:
# hr1.__exit__()

In [22]:
df2 = hr1.find_employee_names("K")

In [23]:
df2

[('Steven', 'King', 'SKING', 100),
 ('Neena', 'Kochhar', 'NKOCHHAR', 101),
 ('Alexander', 'Khoo', 'AKHOO', 115),
 ('Karen', 'Colmenares', 'KCOLMENA', 119),
 ('Payam', 'Kaufling', 'PKAUFLIN', 122),
 ('Kevin', 'Mourgos', 'KMOURGOS', 124),
 ('Irene', 'Mikkilineni', 'IMIKKILI', 126),
 ('Steven', 'Markle', 'SMARKLE', 128),
 ('Mozhe', 'Atkinson', 'MATKINSO', 130),
 ('Ki', 'Gee', 'KGEE', 135),
 ('Hazel', 'Philtanker', 'HPHILTAN', 136),
 ('Renske', 'Ladwig', 'RLADWIG', 137),
 ('Karen', 'Partners', 'KPARTNER', 146),
 ('Eleni', 'Zlotkey', 'EZLOTKEY', 149),
 ('Peter', 'Tucker', 'PTUCKER', 150),
 ('Janette', 'King', 'JKING', 156),
 ('Patrick', 'Sully', 'PSULLY', 157),
 ('Sundita', 'Kumar', 'SKUMAR', 173),
 ('Jack', 'Livingston', 'JLIVINGS', 177),
 ('Kimberely', 'Grant', 'KGRANT', 178),
 ('Kelly', 'Chung', 'KCHUNG', 188),
 ('Randall', 'Perkins', 'RPERKINS', 191),
 ('Kevin', 'Feeney', 'KFEENEY', 197)]

#### Group by

GROUP BY Outside the Database (Functional Programming)

In [24]:
import cx_Oracle
import itertools
from operator import itemgetter

with cx_Oracle.connect("hr/hr@//localhost:1521/XE") as db:
  cursor = db.cursor()
  # fetch all employee data into local variable, no aggregation here
  employees = cursor.execute("select * from employees").fetchall()

D = {}
# department_id is 11-th column
for dept, emp in itertools.groupby(employees, itemgetter(10)):
  D[dept] = len(list(emp))

In [25]:
D

{90: 3,
 60: 5,
 100: 6,
 30: 6,
 50: 20,
 80: 1,
 None: 1,
 10: 1,
 20: 2,
 40: 1,
 70: 1,
 110: 2}

In [28]:
sql_str=""" 
    select job_id,count(*) from employees group by job_id
"""

df = pd.read_sql(sql_str, con=con)

In [29]:
df

,JOB_ID,COUNT(*)
0,AC_ACCOUNT,1
1,AC_MGR,1
2,AD_ASST,1
3,AD_PRES,1
4,AD_VP,2
5,FI_ACCOUNT,5
6,FI_MGR,1
7,HR_REP,1
8,IT_PROG,5
9,MK_MAN,1


In [26]:
jobs = {}
# job_id is 6-th column
for job, emp in itertools.groupby(employees, itemgetter(6)):
  jobs[job] = len(list(emp))

In [27]:
jobs

{'AD_PRES': 1,
 'AD_VP': 2,
 'IT_PROG': 5,
 'FI_MGR': 1,
 'FI_ACCOUNT': 5,
 'PU_MAN': 1,
 'PU_CLERK': 5,
 'ST_MAN': 5,
 'ST_CLERK': 20,
 'SA_MAN': 5,
 'SA_REP': 30,
 'SH_CLERK': 20,
 'AD_ASST': 1,
 'MK_MAN': 1,
 'MK_REP': 1,
 'HR_REP': 1,
 'PR_REP': 1,
 'AC_MGR': 1,
 'AC_ACCOUNT': 1}

### Install APEX on Oracle XE

http://www.oracle.com/technetwork/developer-tools/apex/application-express/upgrade-apex-for-xe-154969.html

Oracle Database 11g Express Edition Release 11.2.0.2.0 - 64bit Production

1) download apex_18.1_en.zip

unzip to APEX_HOME=C:\oracle

cd c:\oracle\apex

2) start sqlplus:

sqlplus /nolog

SQL> CONNECT SYS as SYSDBA

Enter Password:	oracle101

3) Install Application Express:

SQL> @apexins SYSAUX SYSAUX TEMP /i/

Completed Installation in 00:06:06.35


4) Log back into SQL*Plus (as above) and configure the Embedded PL/SQL Gateway (EPG):

SQL> @apex_epg_config.sql C:\oracle

5) Change Application Express password:

SQL> @apxchpwd

email = wen.gong@oracle.com

Enter password for Application Express ADMIN account => "Oracle&101"

6) login to apex via web browser to URL=http://localhost:8080/apex/apex_admin

user/pwd = ADMIN/Oracle&101

### hash

In [111]:
import random
import hashlib
hashlib.sha256(str(random.getrandbits(256)).encode('utf-8')).hexdigest()

'2202ac9ea4be0e2fc737727ca898651e866f17af983bbf806860df76a542d725'